In [29]:
import pandas as pd
import pyarrow.parquet as pq
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [30]:
df_steam_games = pq.read_table('df_steam_games_limpio_final.parquet').to_pandas()

In [31]:
# Selección de columnas relevantes
data = df_steam_games[['id', 'app_name', 'genres', 'tags', 'specs']]

In [32]:
# Limpieza y preprocesamiento de texto
data['combined_features'] = data['genres'] + ' ' + data['tags'] + ' ' + data['specs']

C:\Users\Blasferp\AppData\Local\Temp\ipykernel_15888\2149226059.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['combined_features'] = data['genres'] + ' ' + data['tags'] + ' ' + data['specs']


In [33]:
data.shape

(2020050, 6)

In [34]:
data = data.groupby('id').first().reset_index()

In [35]:
data = data[data['id']!=0]

In [36]:
data['app_name'].nunique()

30708

In [37]:
data = data.sample(n=3500)

In [38]:
data.shape

(3500, 6)

In [39]:
data.reset_index(drop=True, inplace=True)

In [40]:
data

,id,app_name,genres,tags,specs,combined_features
0,665070,visual novel engine export your game,design amp; illustration,design illustration,downloadable content,design amp; illustration design illustration ...
1,512670,winter novel screensaver,casual,indie,single player,casual indie single player
2,417660,sex drive,none,movie,captions available,none movie captions available
3,493280,wallslide,action,early access,single player,action early access single player
4,352140,finders,casual,casual,single player,casual casual single player
...,...,...,...,...,...,...
3495,210068,street fighter x tekken: street fighter swap c...,action,action,single player,action action single player
3496,511410,tiger bunny the movie 2: the rising,none,anime,none,none anime none
3497,259130,wasteland 1 the original classic,adventure,rpg,single player,adventure rpg single player
3498,206091,rocksmith radiohead bodysnatchers,casual,casual,single player,casual casual single player


In [41]:
data.head(10)

,id,app_name,genres,tags,specs,combined_features
0,665070,visual novel engine export your game,design amp; illustration,design illustration,downloadable content,design amp; illustration design illustration ...
1,512670,winter novel screensaver,casual,indie,single player,casual indie single player
2,417660,sex drive,none,movie,captions available,none movie captions available
3,493280,wallslide,action,early access,single player,action early access single player
4,352140,finders,casual,casual,single player,casual casual single player
5,223830,xenonauts,indie,strategy,single player,indie strategy single player
6,678500,the climber,action,adventure,single player,action adventure single player
7,542710,dessert storm,casual,casual,single player,casual casual single player
8,508771,chronicle: runescape legends epic pack,adventure,strategy,single player,adventure strategy single player
9,284990,solarix,action,action,single player,action action single player


In [53]:
data['app_name'].value_counts()

app_name
altitude                                                   2
nasway                                                     1
fsx: steam edition   piper pa 38 tomahawk ii add on        1
rogue singularity                                          1
graviteam tactics: zhalanashkol 1969                       1
                                                          ..
lost in nature                                             1
colorless life                                             1
fantasy grounds   mini dungeon #005: the soularium (5e)    1
army men: toys in space                                    1
disgraced revolutionarys edition dlc                       1
Name: count, Length: 3499, dtype: int64

In [43]:
import pyarrow.parquet as pq
data.to_parquet('df_endopoint6.parquet')

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
CV = CountVectorizer()
converted_metrix = CV.fit_transform(data['combined_features'])

In [46]:
cosine_similarity = cosine_similarity(converted_metrix)

In [48]:
def get_recommendations(app_name, cosine_sim=cosine_similarity ):
    idx = data[data['app_name'] == app_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 10 juegos similares
    game_indices = [i[0] for i in sim_scores]
    return data['app_name'].iloc[game_indices]



In [49]:
# guardo la matriz

import joblib #pickle es alternativa

joblib.dump(cosine_similarity, 'Matriz1.pkl', compress=1)

['Matriz1.pkl']

In [50]:
import joblib
with open('Matriz1.pkl', 'rb') as file:
        modelo = joblib.load(file)

data = pd.read_parquet('df_prueba.parquet')

In [51]:
data.shape

(3500, 6)

In [56]:
recommendations = get_recommendations('lost in nature')
print(recommendations)

11                                 regimental chess
21                           on the path remastered
26    fourchords guitar karaoke   madonna song pack
27                                deus ex: revision
36                                         smogland
Name: app_name, dtype: object
